In [ ]:
# default_exp networks.generators

In [ ]:
#export
import torch.nn as nn
from functools import partial

In [ ]:
#export
class ResBlock(nn.Module):
    def __init__(self, n_feats, kernel_size):
        super(ResBlock, self).__init__()

        self.conv1 = nn.Conv2d(n_feats, n_feats, kernel_size, padding=(kernel_size // 2))
        self.activation = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(n_feats, n_feats, kernel_size, padding=(kernel_size // 2))
        
    def forward(self, input_):
        residual = self.conv1(input_)
        residual = self.activation(residual)
        residual = self.conv2(residual)
        output = input_ + residual

        return output

In [ ]:
#export
class ResNet(nn.Module):
    def __init__(self, in_channels, out_channels, n_feats=64, kernel_size=5, n_resblocks=19):
        super(ResNet, self).__init__()        
        
        self.input_layer = nn.Conv2d(in_channels, n_feats, kernel_size, padding=(kernel_size // 2))
        self.blocks = nn.ModuleList([])
        for _ in range(n_resblocks):
            self.blocks.append(ResBlock(n_feats, kernel_size))
        self.output_layer = nn.Conv2d(n_feats, out_channels, kernel_size, padding=(kernel_size // 2))
        
    def forward(self, input_):
        x = self.input_layer(input_)
        for block in self.blocks:
            x = block(x)
        output = self.output_layer(x)
        return output

In [ ]:
#export
class UpConv2D(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, kernel_size=5, ratio=2):
        super(UpConv2D, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels*(ratio**2), kernel_size, padding=(kernel_size // 2))
        self.upscale = nn.PixelShuffle(ratio)
        
    def forward(self, input_):
        x = self.conv(input_)
        output = self.upscale(x)
        return output

In [ ]:
#export
class MSResNetGenerator(nn.Module):
    def __init__(self, n_scales = 3, n_feats=64, kernel_size=5, n_resblocks=19):
        super(MSResNetGenerator, self).__init__()
        
        # Seungjun does not scale the images down to [-1,1] but rather uses [-127.5, 127.5]
        # this self.mean variable will be used for that. wonder why he calls it the "mean"?
        self.mean = 255.0 / 2
        self.n_scales = n_scales 
        
        coarsest_level = ResNet(in_channels=3, 
                                out_channels=3,
                                n_feats=n_feats,
                                kernel_size=kernel_size,
                                n_resblocks=n_resblocks)
        finer_levels = [ResNet(in_channels=6,
                               out_channels=3, 
                               n_feats=n_feats, 
                               kernel_size=kernel_size, 
                               n_resblocks=n_resblocks) for _ in range(n_scales-1)]
               
        self.scale_networks = nn.ModuleList([]) 
        self.scale_networks.append(coarsest_level)
        self.scale_networks.extend(finer_levels)    
        
        # note that the original implementation always uses 5x5 kernels (default) for upsampling
        self.upconv_blocks = nn.ModuleList([UpConv2D() for _ in range(n_scales-1)])
        
    def forward(self, input_pyramid):
        
        output_pyramid = [None]*self.n_scales
        for scale in range(self.n_scales): 
            if scale == 0:
                input_ = input_pyramid[scale] - self.mean
            else:
                upconvolved_from_previous = self.upconv_blocks[scale-1](output_pyramid[scale-1])
                input_ = torch.cat((input_pyramid[scale] - self.mean, upconvolved_from_previous) ,1)
                
            output_pyramid[scale] = self.scale_networks[scale](input_)
        
        return output_pyramid       

In [ ]:
from skimage.transform import pyramid_gaussian
import numpy as np
from functools import partial
import torch

In [ ]:
img = np.random.rand(32,32,3)
img = img.astype(np.float32)

In [ ]:
pyramid = pyramid_gaussian(img, 2, multichannel=True)

In [ ]:
pyramid = map(np.ascontiguousarray, pyramid)

In [ ]:
transpose = partial(np.transpose, axes=(2,0,1))
pyramid = map(transpose, pyramid)

In [ ]:
pyramid = map(torch.from_numpy, pyramid)

In [ ]:
pyramid = list(map(partial(torch.unsqueeze, dim=0), pyramid))
pyramid = pyramid[::-1]

In [ ]:
net = MSResNetGenerator()

In [ ]:
out = net(pyramid)

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_base_model.ipynb.
Converted 02_generators.ipynb.
Converted 03_discriminators.ipynb.
Converted 99_diffaugment.ipynb.
Converted Untitled.ipynb.
